In [1]:
# import necessary packages
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from keras.models import load_model
import time
import Worker as wk

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-09-27 10:05:14.955604: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-27 10:05:15.234910: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 10:05:16.326540: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2023-09-27 10:05:19.419681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 10:05:19.614271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 10:05:19.614444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
# initialize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

# Load the gesture recognizer model
model = load_model(r'mp_hand_gesture')

# Load class names
f = open(r'gesture.names', 'r')
classNames = f.read().split('\n')
f.close()
print(classNames)

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize volume variables
volume = 0  # Current volume level
volume_step = 10  # Volume change step size

with tf.device('/device:GPU:0'):
    while True:
        time.sleep(0.016)
        # Read each frame from the webcam
        ret, frame = cap.read()
        _, frame = cap.read()

        if not ret:
            # Handle the case where frame capture fails
            # Print an error message or break out of the loop
            print("Failed to capture frame from the webcam")
            break
        x, y, c = frame.shape

        # Flip the frame vertically
        frame = cv2.flip(frame, 1)
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get hand landmark prediction
        result = hands.process(framergb)

        # print(result)

        className = ''

        # post process the result
        if result.multi_hand_landmarks:
            landmarks = []
            for handslms in result.multi_hand_landmarks:
                for lm in handslms.landmark:
                    # print(id, lm)
                    lmx = int(lm.x * x)
                    lmy = int(lm.y * y)

                    landmarks.append([lmx, lmy])

                # Drawing landmarks on frames
                mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

                # Predict gesture
                prediction = model.predict([landmarks])
                # print(prediction)
                classID = np.argmax(prediction)
                className = classNames[classID]
        if className == 'thumbs up':
            volume += volume_step
            wk.Worker(f'amixer set Master {volume}%')
        if className == 'thumbs down':
            volume -= volume_step
            wk.Worker(f'amixer set Master {volume}%')

        # show the prediction on the frame
        cv2.putText(frame, f"{className}{volume})", (10, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 0, 255), 2, cv2.LINE_AA)

        # Show the final output
        cv2.imshow("Output", frame) 

        if cv2.waitKey(1) == ord('q'):
            break

    # release the webcam and destroy all active windows
    cap.release()

    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2023-09-27 10:05:24.354777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 10:05:24.355060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 10:05:24.355215: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com

['okay', 'peace', 'thumbs up', 'thumbs down', 'call me', 'stop', 'rock', 'live long', 'fist', 'smile']


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/piyush/anaconda3/lib/python3.11/site-packages/cv2/qt/plugins"
QObject::moveToThread: Current thread (0xaee05b0) is not the object's thread (0xafcc490).
Cannot move to target thread (0xaee05b0)

QObject::moveToThread: Current thread (0xaee05b0) is not the object's thread (0xafcc490).
Cannot move to target thread (0xaee05b0)

QObject::moveToThread: Current thread (0xaee05b0) is not the object's thread (0xafcc490).
Cannot move to target thread (0xaee05b0)

QObject::moveToThread: Current thread (0xaee05b0) is not the object's thread (0xafcc490).
Cannot move to target thread (0xaee05b0)

QObject::moveToThread: Current thread (0xaee05b0) is not the object's thread (0xafcc490).
Cannot move to target thread (0xaee05b0)

QObject::moveToThread: Current thread (0xaee05b0) is not the object's thread (0xafcc490).
Cannot move to target thread (0xaee05b0)

QObject::moveToThread: Current thread (0xaee05b0) is not the object's thr

1/1 [==============================] - 1s 566ms/step


2023-09-27 10:05:31.102368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


...

1/1 [==============================] - 0s 76ms/step


amixer: invalid option -- '3'
Invalid switch or option -? needs an argument.
amixer: invalid option -- '0'
Invalid switch or option -? needs an argument.
amixer: invalid option -- '%'
Invalid switch or option -? needs an argument.


1/1 [==============================] - 0s 110ms/step


amixer: invalid option -- '4'
Invalid switch or option -? needs an argument.
amixer: invalid option -- '0'
Invalid switch or option -? needs an argument.
amixer: invalid option -- '%'
Invalid switch or option -? needs an argument.


1/1 [==============================] - 0s 63ms/step
